In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymorphy2
from pymorphy2.units.by_lookup import DictionaryAnalyzer
import re
import os
import json
import pickle

from tqdm import tqdm_notebook

%matplotlib inline

# Solver 9

In [2]:
morph = pymorphy2.MorphAnalyzer()
slovarnie_slova = pd.read_csv("../models/dictionaries/slovarnie_slova.txt", header=None).rename({0: "word"}, axis=1)
stress_dict = pd.read_csv("../models/dictionaries/orfoepicheskiy_automatic_gde_udarenie_rf.txt", header=None).rename({0: "word"}, axis=1)
with open("/Users/edgy/Downloads/task_9_dixt_drop_ya.pickle", "rb") as f:
    exact_labels = pickle.load(f)

In [122]:
def solver_9(task, testing=False):
    def is_unverifiable(w):
        for w2 in slovarnie_slova.word:
            if re.match(re.sub(r"\.\.", ".", w), w2):
                return True
        return False

    def is_stressed(w, pos):
        if len(w) == pos:
            w = w[:pos] + w[pos].upper()
        else:
            w = w[:pos] + w[pos].upper() + w[pos+1:]
        return w in stress_dict.word.values

    def word_exists(w):
        analysis = morph.parse(w)
        if (analysis[0].methods_stack[0][0].__class__.__name__ == "DictionaryAnalyzer") and (analysis[0].methods_stack[0][1] == w):
            return True
        return False

    def possible_variants(w):
        amount = 0
        for candidate in "аоеиы":
            w_n = re.sub(r"\.\.", candidate, w)
            analysis = morph.parse(w_n)
            if (analysis[0].methods_stack[0][0].__class__.__name__ == "DictionaryAnalyzer") and (analysis[0].methods_stack[0][1] == w_n):
                amount += 1
        if amount == 0:
            amount = 1
        return amount

    def is_alternant(w):
        #зависящие от конечной согласной корня
        patterns_1 = [
            (r"[а-я]*р\.\.(ст|щ)[а-я]*", "а"),
            (r"[а-я]*р\.\.с[а-су-я]*", "о"),
            (r"[а-я]*л\.\.г[а-я]*", "а"),
            (r"[а-я]*л\.\.ж[а-я]*", "о"),
            (r"[а-я]*ск\.\.к[а-я]*", "а"),
            (r"[а-я]*ск\.\.ч[а-я]*", "о"),
        ]
        #зависящие от суффикса "а" после корня
        patterns_2 = [
            (r"[а-я]*(б|д|м|п|т)\.\.ра[а-я]*", "и"),
            (r"[а-я]*бл\.\.ста[а-я]*", "и"),
            (r"[а-я]*ж\.\.га[а-я]*", "и"),
            (r"[а-я]*ст\.\.ла[а-я]*", "и"),
            (r"[а-я]*ч\.\.та[а-я]*", "и"),
            (r"[а-я]*к\.\.са[а-я]*", "а"),
            (r"[а-я]*(б|д|м|п|т)\.\.р[б-я]*", "е"),
            (r"[а-я]*бл\.\.ст[б-я]*", "е"),
            (r"[а-я]*ж\.\.г[б-я]*", "е"),
            (r"[а-я]*ст\.\.л[б-я]*", "е"),
            (r"[а-я]*ч\.\.т[б-я]*", "е"),
            (r"[а-я]*к\.\.с[б-я]*", "о"),
        ]
        #зависящие от ударения (плов-плав хз почему тут, всегда пишется "а" кроме исключений)
        patterns_3a = [
            (r"[а-я]*з\.\.р[а-я]*", "оа"),
            (r"[а-я]*г\.\.р[а-я]*", "ао"),
            (r"[а-я]*тв\.\.р[а-нп-я]+", "ао"),
        ]
        patterns_3b = [
            (r"[а-я]*пл\.\.в[а-я]*", "оа"),
        ]
        #зависящие от лексического значения
        patterns_4 = [
            (r"[а-я]*м\.\.к[а-я]*", "оа"),
            (r"[а-я]*р\.\.вн[а-я]*", "оа"),
        ]

        exceptions = [
            "росток", "ростов", "ростислав", "ростовщик",
            "отрасль", "скачок", "скачу", "сочетать", "сочетание",
            "чета", "зоревать", "зорянка", "пловец", "пловчиха",
            "плывуны", "уровень", "ровесник", "равнина", "равняйсь", 
            "равнение ",
        ]
        w = w.lower()
        pos_space = re.search(r"\.", w).span()[0]
        for p in patterns_1:
            if re.match(p[0], w):
                for p_i in p[1]:
                    filled_w = re.sub(r"\.\.", p_i, w)
                    if word_exists(filled_w):
                        stressed = is_stressed(filled_w, pos_space)
                        return True, 1, filled_w, pos_space, stressed
        for p in patterns_2:
            if re.match(p[0], w):
                for p_i in p[1]:
                    filled_w = re.sub(r"\.\.", p_i, w)
                    if word_exists(filled_w):
                        stressed = is_stressed(filled_w, pos_space)
                        return True, 2, filled_w, pos_space, stressed
        for p in patterns_4:
            if re.match(p[0], w):
                for p_i in p[1]:
                    filled_w = re.sub(r"\.\.", p_i, w)
                    if word_exists(filled_w):
                        stressed = is_stressed(filled_w, pos_space)
                        return True, 4, filled_w, pos_space, stressed
        for p in patterns_3a:
            if re.match(p[0], w):
                for q, p_i in enumerate(p[1]):
                    filled_w = re.sub(r"\.\.", p_i, w)
                    if word_exists(filled_w):
                        stress_ind = is_stressed(filled_w, pos_space)
                        if (stress_ind) and (q == 0):
                            return True, 3, filled_w, pos_space, True
                        if (q == 1) and (not stress_ind):
                            return True, 3, filled_w, pos_space, False
        for p in patterns_3b:
            if re.match(p[0], w):
                for p_i in p[1]:
                    filled_w = re.sub(r"\.\.", p_i, w)
                    if word_exists(filled_w):
                        stressed = is_stressed(filled_w, pos_space)
                        return True, 3, filled_w, pos_space, stressed
        return False, None, None, pos_space, None

    words = np.array([re.split(r", ", t["text"]) for t in task["question"]["choices"]])
    #обрезаем скобки
#     words = [[re.sub("\([а-я ]+\)", "", t2).strip() for t2 in t1] for t1 in words]
    words = [[re.sub(r"[0-9]+\)", "", re.sub(r"\([а-я ]+\)", "", t2)).strip() for t2 in t1] for t1 in words]
    #в зависимости от числа слов в каждом варианте, мы ожидаем разное число верных ответов
    num_answers = 2
    if len(words[0]) == 1:
        num_answers = 1
    #определяем какой тип нужно искать
    if "чередующ" in task["text"]:
        task_type = 0
    elif "непровер" in task["text"]:
        task_type = 1
    else:
        task_type = 2
    alt_labels = [[is_alternant(t2) for t2 in t1] for t1 in words]
    unver_labels = [[is_unverifiable(t2) for t2 in t1] for t1 in words]
    possible_ways = [[possible_variants(t2) for t2 in t1] for t1 in words]
    scores = np.zeros((len(words), len(words[0]), 3))
    for i in range(scores.shape[0]):
        for j in range(scores.shape[1]):
            scores[i, j, 0] = 0
            scores[i, j, 1] = unver_labels[i][j]
            if alt_labels[i][j][0]:
                scores[i, j, 0] = alt_labels[i][j][0]
            scores[i, j, 2] = 1 - scores[i, j, 0] - 10 * scores[i, j, 1] * (possible_ways[i][j]-1)
    if testing: print(scores)
    agg_scores = scores.mean(axis=1)
    if testing: print(agg_scores)
    agg_scores = agg_scores[:, task_type]
    if testing: print(agg_scores)
    max_score = agg_scores.max()
    second_value = agg_scores[agg_scores.argsort()[-2]]
    answer_numbers = np.arange(len(agg_scores))[agg_scores==max_score]
    if (len(answer_numbers) < 2) and (second_value > 0):
        answer_numbers = np.concatenate([answer_numbers,
                                         np.arange(len(agg_scores))[agg_scores==second_value]])
    answer_numbers = answer_numbers[:2]
#     answer_numbers = agg_scores.argsort()[2:]
    answer_numbers += 1
    answer_numbers = [str(t) for t in answer_numbers]
    return answer_numbers

In [208]:
with open("/Users/edgy/Downloads/task_9/T7009.json", "r") as f:
    js = json.load(f)

In [209]:
js

{'tasks': [{'id': '9',
   'text': 'Укажите варианты ответов, в которых во всех словах одного ряда пропущена безударная проверяемая гласная корня.',
   'meta': {'language': 'ru', 'source': 'yandex_test'},
   'attachments': [],
   'solution': {'correct_variants': [['4', '5'], ['5', '4']]},
   'score': 1,
   'question': {'type': 'multiple_choice',
    'min_choices': 1,
    'choices': [{'id': '1',
      'text': 'зас..вать (поля), разв..вающиеся (страны), попл..вок'},
     {'id': '2', 'text': 'зас..дание, прож..вать (в деревне), сл..гаемые'},
     {'id': '3', 'text': 'озл..бление, погл..щённый, преп..рательства'},
     {'id': '4', 'text': 'с..мволика, зат..вать (драку), кол..бание'},
     {'id': '5', 'text': 'к..робочка, под..конник, впеч..тление'}]}}]}

In [210]:
solver_9(js["tasks"][0], testing=True)

[[[   0.      0.   -100.  ]
  [   0.      0.   -100.  ]
  [   0.      0.   -100.  ]]

 [[   0.      0.      0.9 ]
  [   0.      0.      0.9 ]
  [   0.9     0.      0.45]]

 [[   0.      0.      0.9 ]
  [   0.      0.      1.  ]
  [   0.9     0.      0.45]]

 [[   0.      0.      0.9 ]
  [   0.      0.      0.9 ]
  [   0.      0.      0.9 ]]

 [[   0.      0.      0.9 ]
  [   0.      0.9     0.45]
  [   0.      0.9     0.45]]]
[[   0.            0.         -100.        ]
 [   0.3           0.            0.75      ]
 [   0.3           0.            0.78333333]
 [   0.            0.            0.9       ]
 [   0.            0.6           0.6       ]]
[-100.            0.75          0.78333333    0.9           0.6       ]


['4', '3']

In [126]:
paths_to_tasks = os.listdir("/Users/edgy/Downloads/task_9/")
paths_to_tasks = [t for t in paths_to_tasks if t.startswith("T")]
scores_alt = {"hard": [], "soft": []}
scores_ver = {"hard": [], "soft": []}
scores_unver = {"hard": [], "soft": []}
for path_t in paths_to_tasks:
    with open(f"/Users/edgy/Downloads/task_9/{path_t}", "r") as f:
        js = json.load(f)
    if "correct" in js['tasks'][0]['solution']:
        true_ans = set(js['tasks'][0]['solution']['correct'])
    else: 
        true_ans = set(js['tasks'][0]['solution']['correct_variants'][0])
    answer = set(solver_9(js['tasks'][0]))
    soft_score = len(answer.intersection(true_ans)) / len(answer.union(true_ans))
    hard_score = answer == true_ans
    if "череду" in js['tasks'][0]["text"]:
        scores_alt["soft"].append(soft_score)
        scores_alt["hard"].append(hard_score)
    if "непровер" in js['tasks'][0]["text"]:
        scores_unver["soft"].append(soft_score)
        scores_unver["hard"].append(hard_score)
    else:
        scores_ver["soft"].append(soft_score)
        scores_ver["hard"].append(hard_score)
    if not hard_score:
        print(f"Task {path_t.split('.')[0]}")
        print(js["tasks"][0]["text"])
        print(f"True answer: {true_ans}")
        print(f"Predicted answer: {answer}")

Task T5432
Укажите варианты ответов, в которых во всех словах одного ряда пропущена безударная проверяемая гласная корня.
True answer: {'5', '2'}
Predicted answer: {'3', '2'}
Task T7008
Укажите варианты ответов, в которых во всех словах одного ряда пропущена безударная проверяемая гласная корня.
True answer: {'1', '5'}
Predicted answer: {'1', '3'}
Task T8538
Укажите варианты ответов, в которых во всех словах одного ряда пропущена чередующаяся гласная корня.
True answer: {'5', '2'}
Predicted answer: {'3', '2'}
Task T3749
Укажите варианты ответов, в которых во всех словах одного ряда пропущена безударная проверяемая гласная корня.
True answer: {'3', '4', '2'}
Predicted answer: {'1', '3'}
Task T7012
Укажите варианты ответов, в которых во всех словах одного ряда пропущена безударная проверяемая гласная корня.
True answer: {'1', '4'}
Predicted answer: {'1', '3'}
Task T7013
Укажите варианты ответов, в которых во всех словах одного ряда пропущена безударная непроверяемая гласная корня.
True a

In [127]:
print("Чередующаяся: ", np.mean(scores_alt["soft"]), np.mean(scores_alt["hard"]))
print("Проверяемая: ", np.mean(scores_ver["soft"]), np.mean(scores_ver["hard"]))
print("Непроверяемая: ", np.mean(scores_unver["soft"]), np.mean(scores_unver["hard"]))
print("Total: ", np.mean(scores_unver["soft"]+scores_alt["soft"]+scores_ver["soft"]),
      np.mean(scores_unver["hard"]+scores_alt["hard"]+scores_ver["hard"]))

Чередующаяся:  0.46527777777777773 0.20833333333333334
Проверяемая:  0.44819819819819817 0.16216216216216217
Непроверяемая:  0.6041666666666666 0.4375
Total:  0.4859307359307359 0.23376623376623376


In [214]:
def word_exists(w):
    analysis = morph.parse(w)
    if (analysis[0].methods_stack[0][0].__class__.__name__ == "DictionaryAnalyzer") and\
       (analysis[0].methods_stack[0][1] == w):
        return True
    return False

def solver_9(task, testing=False):
    def is_unverifiable(w):
        for w2 in slovarnie_slova.word:
            if re.match(re.sub(r"[\.]+", ".", w), w2):
                return True
        return False

    def is_stressed(w, pos):
        if len(w) == pos:
            w = w[:pos] + w[pos].upper()
        else:
            w = w[:pos] + w[pos].upper() + w[pos+1:]
        return w in stress_dict.word.values

    def possible_variants(w):
        amount = 0
        for candidate in "аоеиы":
            w_n = re.sub(r"[\.]+", candidate, w)
            if word_exists(w_n):
                amount += 1
        if amount == 0:
            amount = 1
        return amount

    def is_alternant(w):
        #зависящие от конечной согласной корня
        patterns_1 = [
            (r"[а-я]*р[\.]+(ст|щ)[а-я]*", "а"),
            (r"[а-я]*р[\.]+с[а-су-я]*", "о"),
            (r"[а-я]*л[\.]+г[а-я]*", "а"),
            (r"[а-я]*л[\.]+ж[а-я]*", "о"),
            (r"[а-я]*ск[\.]+к[а-я]*", "а"),
            (r"[а-я]*ск[\.]+ч[а-я]*", "о"),
        ]
        #зависящие от суффикса "а" после корня
        patterns_2 = [
            (r"[а-я]*(б|д|м|п|т)[\.]+ра[а-я]*", "и"),
            (r"[а-я]*бл[\.]+ста[а-я]*", "и"),
            (r"[а-я]*ж[\.]+га[а-я]*", "и"),
            (r"[а-я]*ст[\.]+ла[а-я]*", "и"),
            (r"[а-я]*ч[\.]+та[а-я]*", "и"),
            (r"[а-я]*к[\.]+са[а-я]*", "а"),
            (r"[а-я]*(б|д|м|п|т)[\.]+р[б-я]*", "е"),
            (r"[а-я]*бл[\.]+ст[б-я]*", "е"),
            (r"[а-я]*ж[\.]+г[б-я]*", "е"),
            (r"[а-я]*ст[\.]+л[б-я]*", "е"),
            (r"[а-я]*ч[\.]+т[б-я]*", "е"),
            (r"[а-я]*к[\.]+с[б-я]*", "о"),
        ]
        #зависящие от ударения (плов-плав хз почему тут, всегда пишется "а" кроме исключений)
        patterns_3a = [
            (r"[а-я]*з[\.]+р[а-я]*", "оа"),
            (r"[а-я]*г[\.]+р[а-я]*", "ао"),
            (r"[а-я]*тв[\.]+р[а-нп-я]+", "ао"),
        ]
        patterns_3b = [
            (r"[а-я]*пл[\.]+в[а-я]*", "оа"),
        ]
        #зависящие от лексического значения
        patterns_4 = [
            (r"[а-я]*м[\.]+к[а-я]*", "оа"),
            (r"[а-я]*р[\.]+вн[а-я]*", "оа"),
        ]

        exceptions = [
            r"росток", r"ростов", r"ростислав", r"ростовщик",
            r"отрасль", r"скачок", r"скачу", r"сочетать", r"сочетание",
            r"чета", r"зоревать", r"зорянка", r"пловец", r"пловчиха",
            r"плывун[ы]{0,1}", r"уровень", r"ровесник", r"равнина", r"равняйсь", 
            r"равнение ",
        ]
        w = w.lower()
        pos_space = re.search(r"\.", w).span()[0]
        for p in patterns_1:
            if re.match(p[0], w):
                for p_i in p[1]:
                    filled_w = re.sub(r"[\.]+", p_i, w)
                    if word_exists(filled_w):
                        stressed = is_stressed(filled_w, pos_space)
                        return True, 1, filled_w, pos_space, stressed
        for p in patterns_2:
            if re.match(p[0], w):
                for p_i in p[1]:
                    filled_w = re.sub(r"[\.]+", p_i, w)
                    if word_exists(filled_w):
                        stressed = is_stressed(filled_w, pos_space)
                        return True, 2, filled_w, pos_space, stressed
        for p in patterns_4:
            if re.match(p[0], w):
                for p_i in p[1]:
                    filled_w = re.sub(r"[\.]+", p_i, w)
                    if word_exists(filled_w):
                        stressed = is_stressed(filled_w, pos_space)
                        return True, 4, filled_w, pos_space, stressed
        for p in patterns_3a:
            if re.match(p[0], w):
                for q, p_i in enumerate(p[1]):
                    filled_w = re.sub(r"[\.]+", p_i, w)
                    if word_exists(filled_w):
                        stress_ind = is_stressed(filled_w, pos_space)
                        if (stress_ind) and (q == 0):
                            return True, 3, filled_w, pos_space, True
                        if (q == 1) and (not stress_ind):
                            return True, 3, filled_w, pos_space, False
        for p in patterns_3b:
            if re.match(p[0], w):
                for p_i in p[1]:
                    filled_w = re.sub(r"[\.]+", p_i, w)
                    if word_exists(filled_w):
                        stressed = is_stressed(filled_w, pos_space)
                        return True, 3, filled_w, pos_space, stressed
        return False, None, None, pos_space, None

    words = np.array([re.split(r", ", t["text"]) for t in task["question"]["choices"]])
    #обрезаем цифры 
    words = [[re.sub(r"…", "..", re.sub(r"[0-9]+\)", "", t2)).strip() for t2 in t1] for t1 in words]
    exact_types = np.zeros((len(words), len(words[0]))).astype(int)-1
    for i in range(exact_types.shape[0]):
        for j in range(exact_types.shape[1]):
            if words[i][j] in exact_labels["alt"]:
                exact_types[i, j] = 0
            elif words[i][j] in exact_labels["unver"]:
                exact_types[i, j] = 1
            elif words[i][j] in exact_labels["ver"]:
                exact_types[i, j] = 2
    words = [[re.sub(r"\([а-я ]+\)", "", t2).strip() for t2 in t1] for t1 in words]

    #определяем какой тип нужно искать
    if "чередующ" in task["text"]:
        task_type = 0
    elif "непровер" in task["text"]:
        task_type = 1
    else:
        task_type = 2
        
    alt_labels = [[is_alternant(t2) for t2 in t1] for t1 in words]
    unver_labels = [[is_unverifiable(t2) for t2 in t1] for t1 in words]
    possible_ways = [[possible_variants(t2) for t2 in t1] for t1 in words]
    scores = np.zeros((len(words), len(words[0]), 3))
    for i in range(scores.shape[0]):
        for j in range(scores.shape[1]):
            if exact_types[i, j] >= 0:
                if exact_types[i, j] != task_type:
                    scores[i, :, task_type] = -100
                    break
                scores[i, j, exact_types[i, j]] = 1
            else:
                scores[i, j, 0] = 0
                scores[i, j, 1] = unver_labels[i][j] * 0.9
                if alt_labels[i][j][0]:
                    scores[i, j, 0] = alt_labels[i][j][0] * 0.9
                scores[i, j, 2] = 0.9 - 0.5 * (scores[i, j, 0] + scores[i, j, 1])
                
    if testing: print(scores)
    agg_scores = scores.mean(axis=1)
    if testing: print(agg_scores)
    agg_scores = agg_scores[:, task_type]
    if testing: print(agg_scores)
    max_score = agg_scores.max()
    second_value = agg_scores[agg_scores.argsort()[-2]]
    answer_numbers = np.arange(len(agg_scores))[agg_scores==max_score]
    if ((len(answer_numbers) < 2) and (second_value > 0)) or ((len(answer_numbers) == 2) and (second_value > 2.2)):
        answer_numbers = np.concatenate([answer_numbers,
                                         np.arange(len(agg_scores))[agg_scores==second_value]])
        if second_value > 2.2:
            answer_numbers = answer_numbers[:3]
        else:
            answer_numbers = answer_numbers[:2]
    answer_numbers += 1
    answer_numbers = [str(t) for t in answer_numbers]
    return answer_numbers

In [215]:
paths_to_tasks = os.listdir("/Users/edgy/Downloads/task_9/")
paths_to_tasks = [t for t in paths_to_tasks if t.startswith("T")]
scores_alt = {"hard": [], "soft": []}
scores_ver = {"hard": [], "soft": []}
scores_unver = {"hard": [], "soft": []}
for path_t in paths_to_tasks:
    with open(f"/Users/edgy/Downloads/task_9/{path_t}", "r") as f:
        js = json.load(f)
    if "correct" in js['tasks'][0]['solution']:
        true_ans = set(js['tasks'][0]['solution']['correct'])
    else: 
        true_ans = set(js['tasks'][0]['solution']['correct_variants'][0])
    answer = set(solver_9(js['tasks'][0]))
    soft_score = len(answer.intersection(true_ans)) / len(answer.union(true_ans))
    hard_score = answer == true_ans
    if "череду" in js['tasks'][0]["text"]:
        scores_alt["soft"].append(soft_score)
        scores_alt["hard"].append(hard_score)
    if "непровер" in js['tasks'][0]["text"]:
        scores_unver["soft"].append(soft_score)
        scores_unver["hard"].append(hard_score)
    else:
        scores_ver["soft"].append(soft_score)
        scores_ver["hard"].append(hard_score)
    if not hard_score:
        print(f"Task {path_t.split('.')[0]}")
        print(js["tasks"][0]["text"])
        print(f"True answer: {true_ans}")
        print(f"Predicted answer: {answer}")

Task T7009
Укажите варианты ответов, в которых во всех словах одного ряда пропущена безударная проверяемая гласная корня.
True answer: {'5', '4'}
Predicted answer: {'4', '3'}
Task T6903
Укажите варианты ответов, в которых во всех словах одного ряда пропущена безударная чередующаяся гласная корня.
True answer: {'5', '3'}
Predicted answer: {'1', '3'}
Task T6908
Укажите варианты ответов, в которых во всех словах одного ряда пропущена безударная чередующаяся гласная корня.
True answer: {'5', '2'}
Predicted answer: {'5', '4'}
Task T6899
Укажите варианты ответов, в которых во всех словах одного ряда пропущена безударная чередующаяся гласная корня.
True answer: {'1', '4'}
Predicted answer: {'1', '3'}
Task T6909
Укажите варианты ответов, в которых во всех словах одного ряда пропущена безударная проверяемая гласная корня.
True answer: {'1', '4', '3'}
Predicted answer: {'1', '3'}


In [216]:
print("Чередующаяся: ", np.mean(scores_alt["soft"]), np.mean(scores_alt["hard"]))
print("Проверяемая: ", np.mean(scores_ver["soft"]), np.mean(scores_ver["hard"]))
print("Непроверяемая: ", np.mean(scores_unver["soft"]), np.mean(scores_unver["hard"]))
print("Total: ", np.mean(scores_unver["soft"]+scores_alt["soft"]+scores_ver["soft"]),
      np.mean(scores_unver["hard"]+scores_alt["hard"]+scores_ver["hard"]))

Чередующаяся:  0.9166666666666666 0.875
Проверяемая:  0.918918918918919 0.8648648648648649
Непроверяемая:  1.0 1.0
Total:  0.935064935064935 0.8961038961038961


In [8]:
paths_to_tasks = os.listdir("/Users/edgy/Downloads/task_9/")
paths_to_tasks = [t for t in paths_to_tasks if t.startswith("T")]
scores_alt = []
scores_ver = []
scores_unver = []
for path_t in paths_to_tasks:
    with open(f"/Users/edgy/Downloads/task_9/{path_t}", "r") as f:
        js = json.load(f)
    if "correct" in js['tasks'][0]['solution']:
        true_ans = set(js['tasks'][0]['solution']['correct'])
    else: 
        true_ans = set(js['tasks'][0]['solution']['correct_variants'][0])
    answer = {"1", "2", "3", "4", "5"}
    last_score = len(answer.intersection(true_ans)) / len(answer.union(true_ans))
    if "череду" in js['tasks'][0]["text"]:
        scores_alt.append(last_score)
    if "непровер" in js['tasks'][0]["text"]:
        scores_unver.append(last_score)
    else:
        scores_ver.append(last_score)
        
print("Чередующаяся: ", np.mean(scores_alt))
print("Проверяемая: ", np.mean(scores_ver))
print("Непроверяемая: ", np.mean(scores_unver))
print("Total: ", np.mean(scores_unver+scores_alt+scores_ver))

Чередующаяся:  0.4416666666666667
Проверяемая:  0.4540540540540541
Непроверяемая:  0.41250000000000003
Total:  0.44155844155844154


# Solver 17

In [3]:
import sys
import codecs
from keras_bert import load_trained_model_from_checkpoint
import tokenization

# bert_folder = 'multi_cased_L-12_H-768_A-12'
rubert_folder = "/Users/edgy/Downloads/rubert_cased_L-12_H-768_A-12_v2"

Using TensorFlow backend.


In [4]:
# bert_tokenizer = tokenization.FullTokenizer(vocab_file=bert_folder+'/vocab.txt',
#                                             do_lower_case=False)
# bert = load_trained_model_from_checkpoint(bert_folder+'/bert_config.json',
#                                           bert_folder+'/bert_model.ckpt',
#                                           training=True)
rubert_tokenizer = tokenization.FullTokenizer(vocab_file=rubert_folder+'/vocab.txt',
                                              do_lower_case=False)
rubert = load_trained_model_from_checkpoint(rubert_folder+'/bert_config.json',
                                            rubert_folder+'/bert_model.ckpt',
                                            training=True)

W1024 11:26:48.152183 140735762051968 deprecation_wrapper.py:119] From /Users/edgy/projects/ai-journey-2019/notebooks/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W1024 11:26:48.558099 140735762051968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1024 11:26:48.609925 140735762051968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1024 11:26:48.682288 140735762051968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1024 11:26:48.683052 140735762051968 deprecation_wrapper.py:119] 

In [ ]:
def solver_17(task, threshold=0.5, testing=False):
    
    max_length = 512
    
    text = task["text"]
    text = re.sub(r"\(\d\)", "[MASK]", text)
    if testing:
        print(text)
    text = text.replace("[ ]*\[MASK\][ ]*","[MASK]")
    text = text.split("[MASK]")
    
    tokens = ["[CLS]"]
    for i in range(len(text)):
        if i == 0:
            tokens = tokens + tokenizer.tokenize(text[i]) 
        else:
            tokens = tokens + ['[MASK]'] + tokenizer.tokenize(text[i]) 
    tokens = tokens + ['[SEP]'] 
    token_input = tokenizer.convert_tokens_to_ids(tokens)
    token_input = np.array(token_input + [0] * (512 - len(token_input)))
    
    mask_input = np.zeros(max_length)
    mask_input[token_input == 103] = 1
    
    seg_input = np.zeros(max_length)
    predicts = model.predict([token_input.reshape(1, -1), seg_input.reshape(1, -1), mask_input.reshape(1, -1)])[0]
    comma_likelihoods = predicts[0, :, 117][mask_input.astype(bool)]
    dot_likelihoods = predicts[0, :, 119][mask_input.astype(bool)]
    and_likelihoods = predicts[0, :, 549][mask_input.astype(bool)]
    or_likelihoods = predicts[0, :, 10880][mask_input.astype(bool)]
    complex_likelihoods = [t1 + t2 for t1, t2 in zip(comma_likelihoods, and_likelihoods)]
    if testing: print(f"',': {comma_likelihoods}, '.': {dot_likelihoods}, 'и': {and_likelihoods}, 'или': {or_likelihoods}")
    return [str(i+1) for i, t in enumerate(complex_likelihoods) if t >= threshold]

In [166]:
def tmp(task, model=rubert, tokenizer=rubert_tokenizer):
    
    max_length = 512
    
    text = task["text"]
#     for l in "ЙЦУКЕНГШЩЗХЪФЫВАПРОЛДЖЭЁЯЧСМИТЬБЮ":
#         text = re.sub(fr"\n{l}", " "+l.lower(), text)
    text = re.sub(r"\(\d\)", "[MASK]", text)
    text = text.replace("[ ]*\[MASK\][ ]*","[MASK]")
    sentences = re.sub(r"([\.\!\?]+)", "\g<1>$", text)
    sentences = sentences.split("$")
    sentences = [t for t in sentences if "[MASK]" in t]
    preds_list = []
    for sent in sentences:
        sent = sent.split("[MASK]")

        tokens = ["[CLS]"]
        for i in range(len(sent)):
            if i == 0:
                tokens = tokens + tokenizer.tokenize(sent[i]) 
            else:
                tokens = tokens + ['[MASK]'] + tokenizer.tokenize(sent[i]) 
        tokens = tokens + ['[SEP]'] 
        token_input = tokenizer.convert_tokens_to_ids(tokens)
        token_input = np.array(token_input + [0] * (512 - len(token_input)))

        mask_input = np.zeros(max_length)
        mask_token_id = tokenizer.convert_tokens_to_ids(["[MASK]"])[0]
        mask_input[token_input == mask_token_id] = 1

        seg_input = np.zeros(max_length)
        predicts = model.predict([token_input.reshape(1, -1), seg_input.reshape(1, -1), mask_input.reshape(1, -1)])[0]
        predicts = predicts[0, :, :][mask_input.astype(bool)]
        preds_list.append(predicts)
    
    return np.vstack(preds_list)

In [167]:
df2 = pd.DataFrame(columns=["task#", "task_id", "vector", "true"])

for task_number in [17, 18, 19, 20]:
    paths_to_tasks = os.listdir(f"/Users/edgy/Downloads/task_{task_number}/")
    paths_to_tasks = [t for t in paths_to_tasks if t.startswith("T")]
    for path_t in tqdm_notebook(paths_to_tasks):
        with open(f"/Users/edgy/Downloads/task_{task_number}/{path_t}", "r") as f:
            js = json.load(f)
        if "correct" in js['tasks'][0]['solution']:
            true_ans = set(js['tasks'][0]['solution']['correct'])
        else: 
            true_ans = set(js['tasks'][0]['solution']['correct_variants'][0])

        vectors = tmp(js["tasks"][0], model=rubert, tokenizer=rubert_tokenizer)
        for i, _ in enumerate(vectors):
            true = 0
            if str(i + 1) in true_ans:
                true = 1
            df2.loc[len(df2)] = [task_number, path_t, vectors[i], true]

In [140]:
import pickle
import lightgbm
# clf = pickle.load(open("/Users/edgy/Downloads/clf.pickle", "rb"))
lgbm = pickle.load(open("/Users/edgy/Downloads/lgbm.pickle", "rb"))
lgbm1 = pickle.load(open("/Users/edgy/Downloads/lgbm1.pickle", "rb"))
lgbm2 = pickle.load(open("/Users/edgy/Downloads/lgbm2.pickle", "rb"))
lgbm3 = pickle.load(open("/Users/edgy/Downloads/lgbm3.pickle", "rb"))

In [168]:
from sklearn.metrics import accuracy_score, log_loss

In [169]:
df2["pred"] = df2["vector"].apply(lambda x: lgbm.predict_proba(x.reshape(1, -1))[0, 1])
df2["pred1"] = df2["vector"].apply(lambda x: lgbm1.predict_proba(x.reshape(1, -1))[0, 1])
df2["pred2"] = df2["vector"].apply(lambda x: lgbm2.predict_proba(x.reshape(1, -1))[0, 1])
df2["pred3"] = df2["vector"].apply(lambda x: lgbm3.predict_proba(x.reshape(1, -1))[0, 1])

In [170]:
df2["total_pred"] = (df2["pred"]+df2["pred1"]+df2["pred2"]+df2["pred3"]) / 4

In [212]:
d17 = df2[df2["task#"]==17]
thr = 0.5
kek = d17.groupby("task_id", as_index=False).apply(lambda x: accuracy_score(x.true.values.astype(int), x.total_pred.values>thr))
print(sum(kek==1)/len(kek))

thr = 0.6
kek = d17.groupby("task_id", as_index=False).apply(lambda x: accuracy_score(x.true.values.astype(int), np.stack(x.vector.values)[:, comma_id]>thr))
print(sum(kek==1)/len(kek))

0.6612903225806451
0.46774193548387094


In [185]:
d18 = df2[df2["task#"]==18]
thr = 0.1
kek = d18.groupby("task_id", as_index=False).apply(lambda x: accuracy_score(x.true.values.astype(int), x.total_pred.values>thr))
print(sum(kek==1)/len(kek))

thr = 0.1
kek = d18.groupby("task_id", as_index=False).apply(lambda x: accuracy_score(x.true.values.astype(int), np.stack(x.vector.values)[:, comma_id]>thr))
print(sum(kek==1)/len(kek))

0.16666666666666666
0.19047619047619047


In [195]:
d19 = df2[df2["task#"]==19]
thr = 0.7
kek = d19.groupby("task_id", as_index=False).apply(lambda x: accuracy_score(x.true.values.astype(int), x.total_pred.values>thr))
print(sum(kek==1)/len(kek))

thr = 0.6
kek = d19.groupby("task_id", as_index=False).apply(lambda x: accuracy_score(x.true.values.astype(int), np.stack(x.vector.values)[:, comma_id]>thr))
print(sum(kek==1)/len(kek))

0.6935483870967742
0.6290322580645161


In [214]:
d20 = df2[df2["task#"]==20]
thr = 0.75
kek = d20.groupby("task_id", as_index=False).apply(lambda x: accuracy_score(x.true.values.astype(int), x.total_pred.values>thr))
print(sum(kek==1)/len(kek))

thr = 0.7
kek = d20.groupby("task_id", as_index=False).apply(lambda x: accuracy_score(x.true.values.astype(int), np.stack(x.vector.values)[:, comma_id]>thr))
print(sum(kek==1)/len(kek))

0.3870967741935484
0.3387096774193548


In [222]:
tokenizer_bert = rubert_tokenizer
model_bert = rubert

def base_17_18_19_20_new(task):
    max_length = 512
    
    text = task["text"]
    text = re.sub(r"\(\s*\d{1,2}\s*\)", "[MASK]", text)
    sentences = re.sub(r"([\.\!\?]+)", "\g<1>$", text)
    sentences = sentences.split("$")
    sentences = [t for t in sentences if "[MASK]" in t]
    preds_list = []
    for sent in sentences:
        sent = sent.split("[MASK]")

        tokens = ["[CLS]"]
        for i in range(len(sent)):
            if i == 0:
                tokens = tokens + tokenizer_bert.tokenize(sent[i]) 
            else:
                tokens = tokens + ['[MASK]'] + tokenizer_bert.tokenize(sent[i]) 
        tokens = tokens + ['[SEP]'] 
        token_input = tokenizer_bert.convert_tokens_to_ids(tokens)
        token_input = np.array(token_input + [0] * (512 - len(token_input)))

        mask_input = np.zeros(max_length)
        mask_token_id = tokenizer_bert.convert_tokens_to_ids(["[MASK]"])[0]
        mask_input[token_input == mask_token_id] = 1

        seg_input = np.zeros(max_length)
        predicts = model_bert.predict([token_input.reshape(1, -1), seg_input.reshape(1, -1), mask_input.reshape(1, -1)])[0]
        predicts = predicts[0, :, :][mask_input.astype(bool)]
        preds_list.append(predicts)
    
    return np.vstack(preds_list)

def base_17_18_19_20(task):
    max_length = 512

    text = task["text"]
    text = re.sub(r"\(\s*\d{1,2}\s*\)", "[MASK]", text)
    text = text.split("[MASK]")

    tokens = ["[CLS]"]
    for i in range(len(text)):
        if i == 0:
            tokens = tokens + tokenizer_bert.tokenize(text[i].strip())
        else:
            tokens = tokens + ['[MASK]'] + tokenizer_bert.tokenize(text[i].strip())
    tokens = tokens + ['[SEP]']
    token_input = tokenizer_bert.convert_tokens_to_ids(tokens)
    token_input = np.array(token_input + [0] * (max_length - len(token_input)))

    mask_input = np.zeros(max_length)
    mask_input[token_input == 103] = 1

    seg_input = np.zeros(max_length)
    predicts = model_bert.predict([token_input.reshape(1, -1), seg_input.reshape(1, -1), mask_input.reshape(1, -1)])[0]

    comma_id = tokenizer_bert.convert_tokens_to_ids(tokenizer_bert.tokenize(","))[0]
    comma_likelihoods = predicts[0, :, comma_id][mask_input.astype(bool)]
    dot_id = tokenizer_bert.convert_tokens_to_ids(tokenizer_bert.tokenize("."))[0]
    dot_likelihoods = predicts[0, :, dot_id][mask_input.astype(bool)]
    and_id = tokenizer_bert.convert_tokens_to_ids(tokenizer_bert.tokenize("и"))[0]
    and_likelihoods = predicts[0, :, and_id][mask_input.astype(bool)]
    or_id = tokenizer_bert.convert_tokens_to_ids(tokenizer_bert.tokenize("или"))[0]
    or_likelihoods = predicts[0, :, or_id][mask_input.astype(bool)]
    return comma_likelihoods, dot_likelihoods, and_likelihoods, or_likelihoods


def solver_17(task):
    comma_likelihoods, dot_likelihoods, and_likelihoods, or_likelihoods = base_17_18_19_20(task)
    final_preds = comma_likelihoods + and_likelihoods + dot_likelihoods
    return [str(i + 1) for i, t in enumerate(final_preds) if t > 0.65]


def solver_18(task):
    comma_likelihoods, dot_likelihoods, and_likelihoods, or_likelihoods = base_17_18_19_20(task)
    final_preds = comma_likelihoods + and_likelihoods + dot_likelihoods
    return [str(i + 1) for i, t in enumerate(final_preds) if t > 0.48]


def solver_19(task):
    comma_likelihoods, dot_likelihoods, and_likelihoods, or_likelihoods = base_17_18_19_20(task)
    final_preds = comma_likelihoods + or_likelihoods + dot_likelihoods
    return [str(i + 1) for i, t in enumerate(final_preds) if t > 0.62]


def solver_20(task):
    comma_likelihoods, dot_likelihoods, and_likelihoods, or_likelihoods = base_17_18_19_20(task)
    final_preds = comma_likelihoods
    return [str(i + 1) for i, t in enumerate(final_preds) if t > 0.93]

In [223]:
df3 = pd.DataFrame(columns=["task#", "task_id", "base", "new_base", "true"])

for task_number in [17, 18, 19, 20]:
    paths_to_tasks = os.listdir(f"/Users/edgy/Downloads/task_{task_number}/")
    paths_to_tasks = [t for t in paths_to_tasks if t.startswith("T")]
    for path_t in tqdm_notebook(paths_to_tasks):
        with open(f"/Users/edgy/Downloads/task_{task_number}/{path_t}", "r") as f:
            js = json.load(f)
        if "correct" in js['tasks'][0]['solution']:
            true_ans = set(js['tasks'][0]['solution']['correct'])
        else: 
            true_ans = set(js['tasks'][0]['solution']['correct_variants'][0])

        base = base_17_18_19_20(js["tasks"][0])
        new_base = base_17_18_19_20_new(js["tasks"][0])
        df3.loc[len(df3)] = [task_number, path_t, base, new_base, true_ans]

In [224]:
df3

,task#,task_id,base,new_base,true
0,17,T8451.json,"([0.84993994, 0.016140275, 0.010777262, 0.6702...","[[1.4526e-10, 3.3765718e-10, 5.5699145e-10, 2....","{4, 1}"
1,17,T5371.json,"([0.58475184, 0.79958427, 0.13695183, 0.966367...","[[5.131525e-09, 8.051217e-09, 5.6219207e-09, 1...","{5, 4}"
2,17,T6770.json,"([0.9861157, 0.69734395, 0.19565018, 0.1903004...","[[1.2011851e-10, 8.7199685e-11, 1.4016144e-10,...","{6, 1}"
3,17,T618.json,"([0.45829016, 0.52673733, 0.87510985, 0.002094...","[[6.330643e-10, 1.2526201e-09, 9.8928e-10, 5.2...","{3, 2, 5, 1}"
4,17,T2729.json,"([0.8024026, 0.9909046, 0.9888349, 0.99115], [...","[[7.958432e-11, 1.0143078e-10, 1.8929534e-10, ...","{3, 2, 4, 1}"
5,17,T3246.json,"([0.97521645, 0.89513266, 0.011006864, 0.01181...","[[3.8858686e-10, 2.1178259e-10, 2.1681562e-10,...","{2, 1}"
6,17,T593.json,"([0.86831033, 0.06393957, 0.97603613, 0.001958...","[[2.1053128e-11, 4.3016177e-11, 8.72811e-11, 1...","{3, 2, 1}"
7,17,T643.json,"([0.10845462, 0.95771223, 0.9827449, 0.0041633...","[[1.7069686e-09, 1.5322488e-09, 2.784017e-09, ...","{3, 6, 1}"
8,17,T1295.json,"([0.00017139249, 0.7037877, 0.99290246, 0.9212...","[[1.3108816e-09, 8.750849e-09, 3.822795e-09, 6...","{3, 4}"
9,17,T3705.json,"([0.7265231, 0.94102466, 0.49576527, 0.9712977...","[[2.0400865e-09, 1.1012812e-09, 1.098711e-09, ...","{4, 1, 5}"


In [233]:
def f17(base):
    comma_likelihoods, dot_likelihoods, and_likelihoods, or_likelihoods = base
    final_preds = comma_likelihoods + and_likelihoods + dot_likelihoods
    return [str(i + 1) for i, t in enumerate(final_preds) if t > 0.65]

def f18(base):
    comma_likelihoods, dot_likelihoods, and_likelihoods, or_likelihoods = base
    final_preds = comma_likelihoods + and_likelihoods + dot_likelihoods
    return [str(i + 1) for i, t in enumerate(final_preds) if t > 0.48]

def f19(base):
    comma_likelihoods, dot_likelihoods, and_likelihoods, or_likelihoods = base
    final_preds = comma_likelihoods + or_likelihoods + dot_likelihoods
    return [str(i + 1) for i, t in enumerate(final_preds) if t > 0.62]

def f20(base):
    comma_likelihoods, dot_likelihoods, and_likelihoods, or_likelihoods = base
    final_preds = comma_likelihoods
    return [str(i + 1) for i, t in enumerate(final_preds) if t > 0.93]

df3.loc[df3["task#"]==17, "old_ans"] = df3.loc[df3["task#"]==17, "base"].apply(lambda x: set(f17(x)))
df3.loc[df3["task#"]==18, "old_ans"] = df3.loc[df3["task#"]==18, "base"].apply(lambda x: set(f18(x)))
df3.loc[df3["task#"]==19, "old_ans"] = df3.loc[df3["task#"]==19, "base"].apply(lambda x: set(f19(x)))
df3.loc[df3["task#"]==20, "old_ans"] = df3.loc[df3["task#"]==20, "base"].apply(lambda x: set(f20(x)))

In [234]:
df3.groupby("task#").apply(lambda x: np.mean([t1==t2 for t1, t2 in zip(x.true.values, x.old_ans.values)]))

task#
17    0.580645
18    0.214286
19    0.629032
20    0.419355
dtype: float64

In [245]:
def f17_new(base_new):
    vectors = base_new
    predictions = [lgbm.predict_proba(t.reshape(1, -1))[0, 1] for t in vectors]
    ans = [str(i + 1) for i, t in enumerate(predictions) if t > 0.5]
    if len(ans) == 0:
        return [str(np.argmax(predictions)+1)]
    return ans

def f18_new(base_new):
    vectors = base_new
    predictions = [lgbm.predict_proba(t.reshape(1, -1))[0, 1] for t in vectors]
    ans = [str(i + 1) for i, t in enumerate(predictions) if t > 0.1]
    if len(ans) == 0:
        return [str(np.argmax(predictions)+1)]
    return ans

def f19_new(base_new):
    vectors = base_new
    predictions = [lgbm.predict_proba(t.reshape(1, -1))[0, 1] for t in vectors]
    ans = [str(i + 1) for i, t in enumerate(predictions) if t > 0.7]
    if len(ans) == 0:
        return [str(np.argmax(predictions)+1)]
    return ans

def f20_new(base_new):
    vectors = base_new
    predictions = [lgbm.predict_proba(t.reshape(1, -1))[0, 1] for t in vectors]
    ans = [str(i + 1) for i, t in enumerate(predictions) if t > 0.75]
    if len(ans) == 0:
        return [str(np.argmax(predictions)+1)]
    return ans

df3.loc[df3["task#"]==17, "new_ans"] = df3.loc[df3["task#"]==17, "new_base"].apply(lambda x: set(f17_new(x)))
df3.loc[df3["task#"]==18, "new_ans"] = df3.loc[df3["task#"]==18, "new_base"].apply(lambda x: set(f18_new(x)))
df3.loc[df3["task#"]==19, "new_ans"] = df3.loc[df3["task#"]==19, "new_base"].apply(lambda x: set(f19_new(x)))
df3.loc[df3["task#"]==20, "new_ans"] = df3.loc[df3["task#"]==20, "new_base"].apply(lambda x: set(f20_new(x)))

In [246]:
df3.groupby("task#").apply(lambda x: np.mean([t1==t2 for t1, t2 in zip(x.true.values, x.new_ans.values)]))

task#
17    0.661290
18    0.142857
19    0.693548
20    0.387097
dtype: float64

# Solver 14

In [3]:
from keras_bert import load_trained_model_from_checkpoint
import tokenization

rubert_folder = "/Users/edgy/Downloads/rubert_cased_L-12_H-768_A-12_v2"
tokenizer_bert = tokenization.FullTokenizer(vocab_file=rubert_folder+'/vocab.txt',
                                              do_lower_case=False)
bert = load_trained_model_from_checkpoint(rubert_folder+'/bert_config.json',
                                          rubert_folder+'/bert_model.ckpt',
                                          training=True)

Using TensorFlow backend.
W1014 17:35:31.089566 140735601558400 deprecation_wrapper.py:119] From /Users/edgy/projects/ai-journey-2019/notebooks/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W1014 17:35:31.498465 140735601558400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1014 17:35:31.528828 140735601558400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1014 17:35:31.589413 140735601558400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1014 17:35:31.590250 140735601558400 de

In [4]:
def word_exists(w):
    analysis = morph.parse(w)
    if (analysis[0].methods_stack[0][0].__class__.__name__ == "DictionaryAnalyzer") and \
            (analysis[0].methods_stack[0][1] == w):
        return True
    return False

In [5]:
tasks = []
paths_to_tasks = os.listdir("/Users/edgy/Downloads/task_14/")
paths_to_tasks = [t for t in paths_to_tasks if t.startswith("T")]
for path_t in paths_to_tasks:
    with open(f"/Users/edgy/Downloads/task_14/{path_t}", "r") as f:
        js = json.load(f)
        tasks.append(js)

In [6]:
with open("../models/dictionaries/freq_dict_ruscorpora.json", "r") as f:
    freq = json.load(f)

In [21]:
re.sub(r"\ba\b", "kek", "a jnkankjnankj")

'kek jnkankjnankj'

In [58]:
def solver_14(task):
    correct_words = {
        "slitno": ["(на)едине", "(в)последствии", "(буд)то"],
        "razdelno": ["(с)утра", "(по)двое", "(по)трое", "(по)одному", "(под)мышкой"],
        "defis": []
    }
    def possible_variants(w):
        w1 = re.sub(r"[\(\)]", "", w)
        if w.startswith("("):
            w2 = re.sub(r"\)", "-", re.sub(r"\(", "", w))
            w3 = re.sub(r"\)", " ", re.sub(r"\(", "", w))
        else:
            w2 = re.sub(r"\(", "-", re.sub(r"\)", "", w))
            w3 = re.sub(r"\(", " ", re.sub(r"\)", "", w))
        if w.lower() in correct_words["slitno"]:
            return w1, True, w2, False, w3, False
        if w.lower() in correct_words["razdelno"]:
            return w1, False, w2, False, w3, True
        w1_exists = word_exists(w1.lower())
        w2_exists = word_exists(w2.lower())
        w3_exists = word_exists(w3.split(" ")[0].lower()) and word_exists(w3.split(" ")[1].lower())
        # пропишем явно что делать, если слово начинается на ПОЛ, пайморфи тупит
        # в этих случаях, правила приблизительно реализованы (не рассмотрены заглавные,
        # случай с У)
        if w1.startswith("пол"):
            if w1[3] in "леыаояиюэё":
                w1_exists, w2_exists, w3_exists = False, True, False
            else:
                w1_exists, w2_exists, w3_exists = True, False, False
        if (not w2_exists) and (not w3_exists):
            w1_exists = True
        return w1, w1_exists, w2, w2_exists, w3, w3_exists

    def both_together_likelihood(w1_orig, w1_cand, p1, w2_orig, w2_cand, p2, sent):
        max_length = 512
        print("Original sentence: ", sent)
        w1_orig_shield = re.sub("\)", "\\)", re.sub("\(", "\\(", w1_orig))
        w2_orig_shield = re.sub("\)", "\\)", re.sub("\(", "\\(", w2_orig))
        
        if p1 == 1:
            sent = re.sub(w1_orig_shield, w1_cand, sent, 1)
        if p2 == 1:
            sent = re.sub(w2_orig_shield, w2_cand, sent)
        print("With sub: ", sent)
        sent = re.split(fr"({w1_orig_shield}|{w2_orig_shield})", sent)
        sent = [t for t in sent if len(t) > 0]
        tokens = ["[CLS]"]
        exp_tokens = ["[CLS]"]
        word_masks = [0]

        w1_used = False
        if p1 == 1:
            w1_used = True
        for part in sent:
            if (part == w1_orig) and not w1_used:
                kek = tokenizer_bert.tokenize(w1_cand)
                exp_tokens += kek
                tokens += ["[MASK]"] * len(kek)
                word_masks += [1] * len(kek)
                w1_used = True
            elif part == w2_orig:
                kek = tokenizer_bert.tokenize(w2_cand)
                exp_tokens += kek
                tokens += ["[MASK]"] * len(kek)
                word_masks += [2] * len(kek)
            else:
                kek = tokenizer_bert.tokenize(part.strip())
                exp_tokens += kek
                tokens += kek
                word_masks += [0] * len(kek)
        tokens += ["[SEP]"]
        exp_tokens += ["[SEP]"]
        token_input = tokenizer_bert.convert_tokens_to_ids(tokens)
        token_input = np.array(token_input + [0] * (max_length - len(token_input)))
        exp_token_input = tokenizer_bert.convert_tokens_to_ids(exp_tokens)
        exp_token_input = np.array(exp_token_input + [0] * (max_length - len(exp_token_input)))
        word_masks = np.array(word_masks + [0] * (max_length - len(word_masks)))
        mask_input = word_masks != 0
        seg_input = np.zeros(max_length)

        predicts = bert.predict([token_input.reshape(1, -1),
                                 seg_input.reshape(1, -1),
                                 mask_input.reshape(1, -1)])[0]
        preds_1 = predicts[0, word_masks==1]
        exp_token_id_1 = exp_token_input[word_masks==1]
        subprobas_1 = []
        for i, t_id in enumerate(exp_token_id_1):
            subprobas_1.append(preds_1[i, t_id])
        preds_2 = predicts[0, word_masks==2]
        exp_token_id_2 = exp_token_input[word_masks==2]
        subprobas_2 = []
        for i, t_id in enumerate(exp_token_id_2):
            subprobas_2.append(preds_2[i, t_id])
        print(subprobas_1, subprobas_2)
        if p1 == 1:
            return np.mean(subprobas_2)
        if p2 == 1:
            return np.mean(subprobas_1)

    text = task["text"]
    tmp = re.split(r"[\n\.\?\!]+", text)
    sentences = []
    word_pairs = []
    possibilities = []
    together_variants = []
    tmp = [t for t in tmp if len(t) > 0]
    for i, s in enumerate(tmp):
        words = re.findall("[А-ЯЁ]*\([А-ЯЁ]+\)[А-ЯЁ]*", s)
        if (len(words) == 1) and (i < len(tmp)-1):
            words_next = re.findall("[А-ЯЁ]*\([А-ЯЁ]+\)[А-ЯЁ]*", tmp[i+1])
            if len(words_next) == 1:
                tmp[i] = tmp[i] + ". " + tmp[i+1]
                tmp[i+1] = ""
    tmp = [t for t in tmp if len(t) > 0]
    for s in tmp:
        if len(s) < 1:
            continue
        if not s[-1] in ".?!":
            s += "."
        words = re.findall("[А-ЯЁ]*\([А-ЯЁ]+\)[А-ЯЁ]*", s)
        if len(words) == 2:
            w1_1, t1_1, w1_2, t1_2, w1_3, t1_3 = possible_variants(words[0].lower())
            w2_1, t2_1, w2_2, t2_2, w2_3, t2_3 = possible_variants(words[1].lower())
            if t1_1 and t2_1:
                word_pairs.append(words)
                sentences.append(s)
                possibilities.append([[t1_1, t1_2, t1_3], [t2_1, t2_2, t2_3]])
                together_variants.append([w1_1, w2_1])
                if not(t1_2 or t1_3 or t2_2 or t2_3):
                    # У нас есть досрочный ответ
                    return w1_1+w2_1

    max_likelihood = 0
    if len(together_variants) == 1:
        w1_cand, w2_cand = together_variants[0]
        return w1_cand + w2_cand
    elif len(together_variants) == 0:
        return w1_1 + w2_1
    
    for s, word_pair, possibility, together_variant in zip(sentences,
                                                           word_pairs,
                                                           possibilities,
                                                           together_variants):
        w1_orig, w2_orig = word_pair
        w1_cand, w2_cand = together_variant
        likelihood_1 = both_together_likelihood(w1_orig, w1_cand, 0,
                                                w2_orig, w2_cand, 1,
                                                s)
        likelihood_2 = both_together_likelihood(w1_orig, w1_cand, 1,
                                                w2_orig, w2_cand, 0,
                                                s)

        likelihood = min(likelihood_1, likelihood_2)
        if likelihood > max_likelihood:
            max_likelihood = likelihood
            answer = w1_cand + w2_cand
    return answer

In [59]:
scores = []

for i, task in enumerate(tasks):
    print(f"Task # {i}")
    task = task["tasks"][0]
    pred = solver_14(task)
    print(pred)
    if "correct" in task["solution"]:
        answer = task["solution"]["correct"]
        print(answer)
        scores.append(answer == pred)
    else:
        answer = task["solution"]["correct_variants"]
        print(answer)
        scores.append(any([pred == t for t in answer]))

Task # 0
Original sentence:  Группами и (ПО)ОДИНОЧКЕ бродили туристы в курортных шапочках, (И)ТАК было почти весь год.
With sub:  Группами и (ПО)ОДИНОЧКЕ бродили туристы в курортных шапочках, итак было почти весь год.
[0.02149192] []
Original sentence:  Группами и (ПО)ОДИНОЧКЕ бродили туристы в курортных шапочках, (И)ТАК было почти весь год.
With sub:  Группами и поодиночке бродили туристы в курортных шапочках, (И)ТАК было почти весь год.
[] [9.328209e-07, 6.522312e-07]
Original sentence:  (НЕ)СМОТРЯ на прошедшие годы, Николай не смог простить человеку, которого считал другом, его (МАЛО)ДУШИЕ.
With sub:  (НЕ)СМОТРЯ на прошедшие годы, Николай не смог простить человеку, которого считал другом, его малодушие.
[0.0008784544] []
Original sentence:  (НЕ)СМОТРЯ на прошедшие годы, Николай не смог простить человеку, которого считал другом, его (МАЛО)ДУШИЕ.
With sub:  несмотря на прошедшие годы, Николай не смог простить человеку, которого считал другом, его (МАЛО)ДУШИЕ.
[] [0.0006229521, 0.00010

/usr/local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[] [0.6682423]
Original sentence:  Это письмо затерялось, (ПО)ТОМУ что оно ТАК(ЖЕ), как и другая корреспонденция, было отправлено по старому адресу.
With sub:  Это письмо затерялось, (ПО)ТОМУ что оно также, как и другая корреспонденция, было отправлено по старому адресу.
[0.554074] []
Original sentence:  Это письмо затерялось, (ПО)ТОМУ что оно ТАК(ЖЕ), как и другая корреспонденция, было отправлено по старому адресу.
With sub:  Это письмо затерялось, потому что оно ТАК(ЖЕ), как и другая корреспонденция, было отправлено по старому адресу.
[] [0.10584588]
Original sentence:  Отец (ДО)ПОЗДНА сидел в библиотеке, которая была снизу (ДО)ВЕРХУ заполнена книгами.
With sub:  Отец (ДО)ПОЗДНА сидел в библиотеке, которая была снизу доверху заполнена книгами.
[8.1560717e-07, 6.9359207e-06, 0.00053093024] []
Original sentence:  Отец (ДО)ПОЗДНА сидел в библиотеке, которая была снизу (ДО)ВЕРХУ заполнена книгами.
With sub:  Отец допоздна сидел в библиотеке, которая была снизу (ДО)ВЕРХУ заполнена книгами

[0.96533316] []
Original sentence:  Причиной выбрасывания китов на берег могут быть гидролокаторы военных, достаточно мощные, ЧТО(БЫ) проникнуть (В)ГЛУБЬ океана и напугать животных.
With sub:  Причиной выбрасывания китов на берег могут быть гидролокаторы военных, достаточно мощные, чтобы проникнуть (В)ГЛУБЬ океана и напугать животных.
[] [0.20886372]
Original sentence:  Недолго пройдя в темноте, Костя понял, что (СО)ВСЕМ сбился с пути, наверно, ТО(ЖЕ) понял и командир.
With sub:  Недолго пройдя в темноте, Костя понял, что (СО)ВСЕМ сбился с пути, наверно, тоже понял и командир.
[0.0003939438] []
Original sentence:  Недолго пройдя в темноте, Костя понял, что (СО)ВСЕМ сбился с пути, наверно, ТО(ЖЕ) понял и командир.
With sub:  Недолго пройдя в темноте, Костя понял, что совсем сбился с пути, наверно, ТО(ЖЕ) понял и командир.
[] [0.00061808823]
Original sentence:  Новобранцев до принятия присяги не полагалось отпускать в город (ПО)ОДИНОЧКЕ, но инструктор, (В)ВИДУ моего необычайного успеха п

[0.052072737] []
Original sentence:  В дальнем конце коридора стояло странное сооружение, (НА)ПОДОБИЕ огромной рельсы, уходящей под углом (В)ВЕРХ.
With sub:  В дальнем конце коридора стояло странное сооружение, наподобие огромной рельсы, уходящей под углом (В)ВЕРХ.
[] [0.055349145]
Original sentence:  Всё ТАК(ЖЕ) горела сальная свеча на столе, росли тихие герани на подоконниках, тикали часы, (ПРИ)ЧЁМ звук их с годами совсем не изменился.
With sub:  Всё ТАК(ЖЕ) горела сальная свеча на столе, росли тихие герани на подоконниках, тикали часы, причём звук их с годами совсем не изменился.
[0.00076193264] []
Original sentence:  Всё ТАК(ЖЕ) горела сальная свеча на столе, росли тихие герани на подоконниках, тикали часы, (ПРИ)ЧЁМ звук их с годами совсем не изменился.
With sub:  Всё также горела сальная свеча на столе, росли тихие герани на подоконниках, тикали часы, (ПРИ)ЧЁМ звук их с годами совсем не изменился.
[] [0.014302498]
наподобиевверх
['наподобиевверх', 'вверхнаподобие']
Task # 21
Origi

[] [6.8076674e-06, 8.4491575e-07, 0.00040847473]
термокостюмынесмотря
['термокостюмынесмотря', 'несмотрятермокостюмы']
Task # 27
полшкафаполупустой
['полшкафаполупустой', 'полупустойполшкафа']
Task # 28
Original sentence:  Во всём городе не было людей, настроенных ТАК(ЖЕ) спокойно и в ТО(ЖЕ) время торжественно, как эти двое.
With sub:  Во всём городе не было людей, настроенных ТАК(ЖЕ) спокойно и в тоже время торжественно, как эти двое.
[0.0011781092] []
Original sentence:  Во всём городе не было людей, настроенных ТАК(ЖЕ) спокойно и в ТО(ЖЕ) время торжественно, как эти двое.
With sub:  Во всём городе не было людей, настроенных также спокойно и в ТО(ЖЕ) время торжественно, как эти двое.
[] [0.007951428]
Original sentence:  (С)НАЧАЛА лучше сделать самые трудные уроки, (ПО)ТОМУ что их выполнение потребует больше времени.
With sub:  (С)НАЧАЛА лучше сделать самые трудные уроки, потому что их выполнение потребует больше времени.
[7.6909096e-08] []
Original sentence:  (С)НАЧАЛА лучше сделать 

[0.80918974] []
Original sentence:  Но он ничего не сделал, (ПО)ТОМУ что и не мог ничего сделать, просто не было у него ничего такого затаённого, ЧТО(Б) он мог вытащить наружу.
With sub:  Но он ничего не сделал, потому что и не мог ничего сделать, просто не было у него ничего такого затаённого, ЧТО(Б) он мог вытащить наружу.
[] [0.002653919]
потомучтоб
['оттогоподолгу', 'подолгуоттого']
Task # 38
Original sentence:  Нет, судите наш народ не (ПО)ТОМУ, что он есть, а (ПО)ТОМУ, чем он желал бы стать.
With sub:  Нет, судите наш народ не потому, что он есть, а потому, чем он желал бы стать.
[] []
Original sentence:  Нет, судите наш народ не (ПО)ТОМУ, что он есть, а (ПО)ТОМУ, чем он желал бы стать.
With sub:  Нет, судите наш народ не потому, что он есть, а (ПО)ТОМУ, чем он желал бы стать.
[] [0.99703646]
Original sentence:  (В)ПОСЛЕДСТВИИ речь автора была проста, (ПРИ)ЧЁМ выразительна.
With sub:  (В)ПОСЛЕДСТВИИ речь автора была проста, причём выразительна.
[1.7176798e-07] []
Original sentenc

[0.8170229] []
Original sentence:  Собранные растения перед сушкой следует очистить от земли, а ТАК(ЖЕ) от посторонних примесей, (ЗА)ТЕМ разложить их на подстилке.
With sub:  Собранные растения перед сушкой следует очистить от земли, а также от посторонних примесей, (ЗА)ТЕМ разложить их на подстилке.
[] [0.2139365]
Original sentence:  Мои спутники ТО(ЖЕ) осматривали берег, но (В)ВИДУ имели совсем другое.
With sub:  Мои спутники ТО(ЖЕ) осматривали берег, но ввиду имели совсем другое.
[0.038514152] []
Original sentence:  Мои спутники ТО(ЖЕ) осматривали берег, но (В)ВИДУ имели совсем другое.
With sub:  Мои спутники тоже осматривали берег, но (В)ВИДУ имели совсем другое.
[] [0.0008291117]
Original sentence:  (ВО)ВРЕМЯ занятия все задания мы сделали (ВО)ВРЕМЯ.
With sub:  вовремя занятия все задания мы сделали вовремя.
[] []
Original sentence:  (ВО)ВРЕМЯ занятия все задания мы сделали (ВО)ВРЕМЯ.
With sub:  вовремя занятия все задания мы сделали (ВО)ВРЕМЯ.
[] [0.011021955]
такжезатем
['такжез

[] [0.1008214]
Original sentence:  Анна готова была много работать, ЧТО(БЫ) не оставаться (НА)ЕДИНЕ с грустными мыслями.
With sub:  Анна готова была много работать, ЧТО(БЫ) не оставаться наедине с грустными мыслями.
[0.33998922] []
Original sentence:  Анна готова была много работать, ЧТО(БЫ) не оставаться (НА)ЕДИНЕ с грустными мыслями.
With sub:  Анна готова была много работать, чтобы не оставаться (НА)ЕДИНЕ с грустными мыслями.
[] [0.6733329]
чтобынаедине
чтобынаедине


In [60]:
np.mean(scores)

0.6730769230769231

In [36]:
t = "Группами и (ПО)ОДИНОЧКЕ бродили туристы в курортных шапочках, (И)ТАК было почти весь год."
re.sub("\(ПО\)ОДИНОЧКЕ", "поодиночке", t)

'Группами и (ПО)ОДИНОЧКЕ бродили туристы в курортных шапочках, (И)ТАК было почти весь год.'

In [299]:
tasks[35]["tasks"][0]

{'id': '14',
 'text': 'Определите предложение, в котором оба выделенных слова пишутся слитно.\nПриходилось следить (ЗА)ТЕМ, ЧТО(БЫ) выдерживать определённую дистанцию.\nБедному псу, (ПО)ВИДИМОМУ, ТО(ЖЕ) становилось страшно.\nЧТО(БЫ) почитать, ЧТО(БЫ) больше знать об истории Рима.\n(ЗА)ЧЕМ пойдёшь, ТО(И) найдёшь.\n(ОТ)ЧЕГО лентяй гладок? (ОТ)ТОГО, что на отдых падок.\nРаскройте скобки и выпишите эти два слова.',
 'meta': {'language': 'ru', 'source': 'yandex_test'},
 'attachments': [],
 'solution': {'correct_variants': ['отчегооттого', 'оттогоотчего']},
 'score': 1,
 'question': {'type': 'text',
  'max_length': 30,
  'recommended_length': 20,
  'restriction': 'word'}}

In [304]:
solver_14(tasks[35]["tasks"][0])

['(ЗА)ТЕМ', 'ЧТО(БЫ)']
['(ПО)ВИДИМОМУ', 'ТО(ЖЕ)']
['ЧТО(БЫ)', 'ЧТО(БЫ)']
['(ЗА)ЧЕМ', 'ТО(И)']
['(ОТ)ЧЕГО', '(ОТ)ТОГО']
Original sentence:  Приходилось следить (ЗА)ТЕМ, ЧТО(БЫ) выдерживать определённую дистанцию.
With sub:  Приходилось следить (ЗА)ТЕМ, чтобы выдерживать определённую дистанцию.
[0.00025591816] 1 [] 1
Original sentence:  Приходилось следить (ЗА)ТЕМ, ЧТО(БЫ) выдерживать определённую дистанцию.
With sub:  Приходилось следить затем, ЧТО(БЫ) выдерживать определённую дистанцию.
[] 1 [0.81290406] 1
Original sentence:  ЧТО(БЫ) почитать, ЧТО(БЫ) больше знать об истории Рима.
With sub:  чтобы почитать, чтобы больше знать об истории Рима.
[] 1 [] 1
Original sentence:  ЧТО(БЫ) почитать, ЧТО(БЫ) больше знать об истории Рима.
With sub:  чтобы почитать, чтобы больше знать об истории Рима.
[] 1 [] 1
Original sentence:  (ЗА)ЧЕМ пойдёшь, ТО(И) найдёшь.
With sub:  (ЗА)ЧЕМ пойдёшь, тои найдёшь.
[7.02874e-07] 1 [] 1
Original sentence:  (ЗА)ЧЕМ пойдёшь, ТО(И) найдёшь.
With sub:  зачем пойдёшь

'отчегооттого'

In [254]:
scores_not = []

for i, task in enumerate(tasks):
    print(f"Task # {i}")
    task = task["tasks"][0]
    pred = solver_14(task)
    print(pred)
    if "correct" in task["solution"]:
        answer = task["solution"]["correct"]
        print(answer)
        scores_not.append(answer == pred)
    else:
        answer = task["solution"]["correct_variants"]
        print(answer)
        scores_not.append(any([pred == t for t in answer]))

Task # 0
['Группами и ', '(ПО)ОДИНОЧКЕ', ' бродили туристы в курортных шапочках, ', '(И)ТАК', ' было почти весь год.']
[0.021590982] 1 [9.774301e-07, 7.4745697e-07] 1
8.62443528149015e-07
['(НЕ)СМОТРЯ', ' на прошедшие годы, Николай не смог простить человеку, которого считал другом, его ', '(МАЛО)ДУШИЕ', '.']
[0.0006111483] 1 [0.00077143055, 0.00013530701, 2.719771e-05] 1
0.0003113117709290236
['(С)НАЧАЛА', ' дети шли в колонне ', '(ПО)ДВОЕ', ', но потом стали двигаться беспорядочными кучками.']
[0.00016406883] 1 [2.819315e-06, 1.4796928e-05] 1
8.808121492620558e-06
['Я всё ', 'ТАК(ЖЕ)', ' очень жалел о сказанном, хотел забыть обо всём, ', '(ПРИ)ТОМ', ' как можно скорее.']
[0.00041144894] 1 [1.7098411e-05] 1
1.7098411262850277e-05
несмотрямалодушие
['несмотрямалодушие', 'малодушиенесмотря']
Task # 1
['Собираясь ', '(НА)ВСТРЕЧУ', ' с представителем фирмы, парень волновался, ', '(НЕ)СМОТРЯ', ' на большой опыт работы.']
[3.4215332e-06] 1 [0.71765965] 1
3.4215331652376335e-06
['(ВО)ВРЕМЯ', 

['Теперь, даже ', '(НЕ)СМОТРЯ', ' на седину, морщины и очки, его ', '(НА)ЧИСТО', ' лишённое эмоций лицо кажется прекрасным.']
[0.98087686] 1 [4.838296e-07, 0.0001007038] 1
5.059381510363892e-05
['Илье Антонычу наскучило носить ружьё ', '(ПОД)МЫШКОЙ', ', ', '(ПО)ЭТОМУ', ' он перекинул его на плечо.']
[7.894238e-06, 0.00012953689, 0.003978688] 1 [0.00182377] 1
0.0013720396673306823
['(НА)КОНЕЦ', ' прошла неделя, а комната всё ', 'ТАК(ЖЕ)', ' была заперта.']
[9.930327e-07] 1 [0.0006739774] 1
9.930326996254735e-07
подмышкойпоэтому
['несмотряначисто', 'начистонесмотря']
Task # 17
подальшевмиг
['подальшевмиг', 'вмигподальше']
Task # 18
['ТО(ЖЕ)', ' тихое мерцание зеленоватого прозрачного неба, ', 'ТАК(ЖЕ)', ' тянет с реки холодной влагой.']
[1.4368223e-07] 1 [7.5075775e-05] 1
1.4368222878147208e-07
['(ПОЛ)ДНЯ', ' ушло на сборы и подготовку к выступлению, ', '(ПРИ)ЧЁМ', ' оказалось, что ещё далеко не всё готово.']
[6.3048516e-05, 1.6333253e-05] 1 [0.0010269227] 1
3.969088356825523e-05
полдняп

['Нет, судите наш народ не ', '(ПО)ТОМУ', ', что он есть, а ', '(ПО)ТОМУ', ', чем он желал бы стать.']
[0.461019] 1 [0.046049424] 1
0.04604942351579666
['(В)ПОСЛЕДСТВИИ', ' речь автора была проста, ', '(ПРИ)ЧЁМ', ' выразительна.']
[1.2495465e-07] 1 [6.4830434e-05] 1
1.2495465284700913e-07
потомупотому
['впоследствиипричём', 'причёмвпоследствии']
Task # 39
['Мы отправили посылку ', '(В)НАЧАЛЕ', ' месяца, и она пришла ', '(ВО)ВРЕМЯ', '.']
[7.671429e-05] 1 [0.0683443] 1
7.67142919357866e-05
['Во ', 'ЧТО(БЫ)', ' нам поиграть, ', 'ЧТО(БЫ)', ' весело провести время.']
[6.4696e-06] 1 [0.5206495] 1
6.469600066338899e-06
['Идите ', '(В)НИЗ', ' по склону, там и будет ', '(АВТО)СТОЯНКА', '.']
[0.54001874] 1 [1.2882702e-05, 0.023408066] 1
0.011710474267601967
внизавтостоянка
['внизавтостоянка', 'автостоянкавниз']
Task # 40
полгодазамуж
['полгодазамуж', 'замужполгода']
Task # 41
['«И имейте ', '(В)ВИДУ', ': ', 'ЧТО(БЫ)', ' там ещё вы ни придумали, мне всё станет известно.']
[0.023901682] 1 [0.00062

In [278]:
np.mean(scores_not)

0.5961538461538461

In [280]:
for t1, t2 in zip(scores, scores_not):
    print(t1, t2)

True True
False False
True True
False False
False False
False False
True True
False False
True True
True True
True True
True True
True True
False False
False False
False False
False False
True True
True True
True True
True True
False False
True True
False False
True True
True True
True True
True True
False False
True True
True True
False False
True True
True True
False False
False False
True True
False False
True False
True True
True True
False False
True True
False False
True True
True True
True True
False False
False False
True True
True True
True True
